In [15]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import evaluate

import torch

import nltk
nltk.download('punkt')

from transformers import (MBartForConditionalGeneration, 
                          MBartTokenizer, 
                          DataCollatorForSeq2Seq,
                          MT5ForConditionalGeneration,
                          MT5Tokenizer,
                          pipeline,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)

from datasets import load_dataset

[nltk_data] Downloading package punkt to /home/sanya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Настройка

In [16]:
# Параметры Bart токенайзера
MAX_LENGTH = 600
PADDING = 'max_length'
TRUNCATION = True
RETURN_TENSORS = 'pt'

# Параметры Bart
NO_REPEAT_NGRAM = 4

# Параметры T5 токенайзера
MAX_TARGET_TOKENS_COUNT = 128
MAX_SOURCE_TOKENS_COUNT = 1024

# Параметры T5
OUTPUT_DIR = "T5"
EVALUATION_STRATEGY = "steps"
EVAL_STEPS = 25
LOGGING_STEPS = 25
LEARNING_RATE = 4e-4
PER_DEVICE_TRAIN_BATCH_SIZE = 2
PER_DEVICE_EVAL_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 64
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIMIT = 3
NUM_TRAIN_EPOCHS = 1
FP16 = False
PREDICT_WITH_GENERATE = True
GENERATION_MAX_LENGTH = MAX_TARGET_TOKENS_COUNT
GENERATION_NUM_BEAMS = 5

### Устройство для обучения

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Метрики

In [18]:
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")

# Загрузка датасета

In [19]:
dataset = load_dataset('IlyaGusev/gazeta')
dataset['train'] = dataset['train'].select(range(200))

In [20]:
df = pd.DataFrame(dataset['test'])
df

,text,summary,title,date,url
0,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,2020-09-01 00:22:59,https://www.gazeta.ru/politics/2020/08/31_a_13...
1,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,2020-09-01 08:08:16,https://www.gazeta.ru/politics/2020/08/31_a_13...
2,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,2020-09-01 09:21:38,https://www.gazeta.ru/politics/2020/09/01_a_13...
3,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает крайне опасными действия США на...,2020-09-01 09:33:30,https://www.gazeta.ru/army/2020/09/01/13222904...
4,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,В России вступил в силу закон о внесудебном ба...,2020-09-01 09:49:24,https://www.gazeta.ru/business/2020/09/01/1322...
...,...,...,...,...,...
6788,Феерическим взрывом в атмосфере закончился пер...,Первый пуск американской частной ракеты Alpha ...,СМИ назвали успехом запуск частной ракеты Alph...,2021-09-03 09:28:34,https://www.gazeta.ru/science/2021/09/03_a_139...
6789,Глава Минвостокразвития Алексей Чекунков 31 ав...,Городская агломерация Владивостока может дости...,На ВЭФ подписано соглашение о создании города ...,2021-09-03 09:45:36,https://www.gazeta.ru/economics/2021/09/03/139...
6790,После взрывов во Врбетице отношения между Чехи...,Из-за геополитической и экономической роли Рос...,Глава чешского МИДа призвал восстанавливать св...,2021-09-03 10:44:03,https://www.gazeta.ru/politics/2021/09/03_a_13...
6791,Астрономы впервые наблюдали процесс вспышки св...,Международная группа астрономов впервые наблюд...,Астрономы наблюдали вспышку сверхновой нового ...,2021-09-03 11:43:50,https://www.gazeta.ru/science/2021/09/03_a_139...


# Тестирование Bart

In [7]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"

In [8]:
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

In [9]:
predicts = []

for text in tqdm(dataset['test']['text']):

    input_ids = tokenizer(text, 
                          max_length = MAX_LENGTH,
                          padding = PADDING,
                          truncation = TRUNCATION,
                          return_tensors = RETURN_TENSORS)
    
    input_ids = input_ids['input_ids'].to(device)
    
    output_ids = model.generate(input_ids = input_ids,
                                no_repeat_ngram_size = NO_REPEAT_NGRAM)
    
    output_ids = output_ids[0]

    predicts.append(tokenizer.decode(output_ids, skip_special_tokens = True))

100%|██████████| 100/100 [01:28<00:00,  1.13it/s]


In [10]:
df['summary'][10]

'Лишний вес при COVID-19 повышает риск столкнуться с осложнениями и оказаться на ИВЛ, предупреждают французские врачи — ожирение наблюдается почти у всех пациентов с коронавирусом, попавших в отделения интенсивной терапии. И чем выше вес, тем выше и вероятность пострадать от тяжелого течения болезни.'

In [11]:
predicts[10]

'Ожирение повышает риск тяжелого течения COVID-19, предупреждают врачи из Лилльского университета во Франции. По их данным, среди пациентов с лишним весом почти половина страдала от ожирения, у четверти оно было тяжелым.'

In [12]:
rouge_res = rouge.compute(predictions = predicts,
                          references = df['summary'].values)

bleu_res = bleu.compute(predictions = predicts,
                        references = df['summary'].values)

In [13]:
print(f'Rouge: {rouge_res}')
print(f'Bleu = {bleu_res["bleu"]:.2f}')

Rouge: {'rouge1': 0.25378338328338335, 'rouge2': 0.07744572094572094, 'rougeL': 0.2443589743589744, 'rougeLsum': 0.24598917748917754}
Bleu = 0.13


# Тестирование T5

In [21]:
model_name = "google/mt5-small"

In [22]:
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = MT5Tokenizer.from_pretrained(model_name)

In [23]:
def preprocess(examples):
   
    inputs = examples['text']

    model_inputs = tokenizer(inputs, 
                             max_length = MAX_SOURCE_TOKENS_COUNT, 
                             truncation = True)
    
    labels = tokenizer(text_target = examples['summary'], 
                       max_length = MAX_TARGET_TOKENS_COUNT,
                       truncation = True)
    
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [24]:
tokenized_dataset = dataset.map(preprocess, batched = True)

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model)

In [26]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = {}
    result_rouge = rouge.compute(predictions = decoded_preds, references = decoded_labels)
    
    # Extract a few results
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})
    
    result_bleu = bleu.compute(predictions = decoded_preds, references = decoded_labels)
    
    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])
    
    return {k: round(v, 4) for k, v in result.items()}

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir = OUTPUT_DIR,
    evaluation_strategy = EVALUATION_STRATEGY,
    eval_steps = EVAL_STEPS,
    logging_steps = LOGGING_STEPS,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size = PER_DEVICE_EVAL_BATCH_SIZE,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    weight_decay = WEIGHT_DECAY,
    save_total_limit = SAVE_TOTAL_LIMIT,
    num_train_epochs = NUM_TRAIN_EPOCHS,
    fp16 = FP16,
    predict_with_generate = PREDICT_WITH_GENERATE,
    generation_max_length = GENERATION_MAX_LENGTH,
    generation_num_beams = GENERATION_NUM_BEAMS
)

In [28]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

In [29]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

{'train_runtime': 10.5903, 'train_samples_per_second': 18.885, 'train_steps_per_second': 0.094, 'train_loss': 21.674280166625977, 'epoch': 0.64}


TrainOutput(global_step=1, training_loss=21.674280166625977, metrics={'train_runtime': 10.5903, 'train_samples_per_second': 18.885, 'train_steps_per_second': 0.094, 'train_loss': 21.674280166625977, 'epoch': 0.64})

In [30]:
trainer.evaluate()

  0%|          | 0/3397 [00:00<?, ?it/s]